In [2]:
import os
import gradio as gr
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize LLM (Groq LLaMA3)
llm = ChatGroq(
    temperature=0.3,
    model_name="llama3-8b-8192"
)

# Few-shot + Chain-of-Thoughts prompt template for joke analysis
prompt = PromptTemplate(
    input_variables=["joke"],
    template="""
You are a witty and thoughtful AI. When given a joke, determine if it's funny or not. If it is funny, explain why using step-by-step reasoning. If it's not funny, explain why it might fail to be humorous. Use examples as reference.

Example 1:
Joke: Why did the scarecrow win an award? Because he was outstanding in his field.
Answer: Funny ✅
Reasoning: It's a clever play on words. "Outstanding in his field" can mean both literally standing in a field and being excellent at what he does. This pun creates humor.

Example 2:
Joke: I used to be a banker but I lost interest.
Answer: Funny ✅
Reasoning: This is a pun. "Lost interest" refers to both emotional disinterest and financial interest, making it witty.

Example 3:
Joke: I have a dog.
Answer: Not Funny ❌
Reasoning: This statement lacks a punchline or surprise, which are key elements of humor.

Now analyze this joke:
Joke: {joke}
Answer:
"""
)

# Create runnable chain
chain = prompt | llm | StrOutputParser()

# Function to evaluate the joke
def analyze_joke(joke):
    return chain.invoke({"joke": joke}).strip()

# Gradio interface with styling
with gr.Blocks(css="""
body {
    background-color: #f6f8fa;
    font-family: 'Segoe UI', sans-serif;
    color: #1c1e21;
}
#joke-input textarea {
    font-size: 15px;
    padding: 14px;
    border-radius: 10px;
    border: 1px solid #ccc;
    background-color: #fff;
    color: #000;
}
#submit-btn {
    background-color: #6c63ff;
    color: white;
    padding: 12px 20px;
    border: none;
    font-size: 16px;
    border-radius: 8px;
    margin-top: 12px;
    transition: background-color 0.3s ease;
}
#submit-btn:hover {
    background-color: #5548d9;
}
#joke-output textarea {
    font-size: 15px;
    background-color: #fdfdfd;
    padding: 14px;
    border-radius: 10px;
    color: #000;
    border: 1px solid #ccc;
}
""") as demo:

    gr.Markdown("""
    <div style='text-align:center;'>
        <h1 style='color:#6c63ff;'>😂 Joke Analyzer: Is it Funny or Not?</h1>
        <p>Enter a joke. This assistant will analyze whether it's funny and explain why using few-shot reasoning and chain-of-thoughts.</p>
    </div>
    """)

    with gr.Row():
        with gr.Column():
            joke_input = gr.Textbox(
                label="🃏 Your Joke",
                lines=3,
                elem_id="joke-input",
                placeholder="E.g. Why don’t skeletons fight each other? They don’t have the guts.",
                show_copy_button=True
            )
            run_button = gr.Button("🧠 Analyze Joke", elem_id="submit-btn")

        with gr.Column():
            joke_output = gr.Textbox(
                label="🎯 Joke Analysis",
                lines=10,
                elem_id="joke-output",
                interactive=True,
                show_copy_button=True
            )

    run_button.click(fn=analyze_joke, inputs=joke_input, outputs=joke_output)

# Launch Gradio app
demo.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
